In [2]:
from rdkit import Chem
from rdkit.Chem import PandasTools
import pandas as pd
import numpy as np

Sanitizando dados do Coconut

In [3]:
%%time

suppl = Chem.SDMolSupplier('coconut_05_2022.sdf')

coco = []

for mol in suppl:
    if type(mol) == Chem.rdchem.Mol:
        coco.append(mol.GetPropsAsDict())

RDKit ERROR: [15:01:32] Explicit valence for atom # 20 C, 6, is greater than permitted
RDKit ERROR: [15:01:32] ERROR: Could not sanitize molecule ending on line 53
[15:01:32] Explicit valence for atom # 20 C, 6, is greater than permitted
[15:01:32] ERROR: Could not sanitize molecule ending on line 53
RDKit ERROR: [15:01:32] ERROR: Explicit valence for atom # 20 C, 6, is greater than permitted
[15:01:32] ERROR: Explicit valence for atom # 20 C, 6, is greater than permitted
[15:01:34] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [15:01:34] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [15:01:34] ERROR: Could not sanitize molecule ending on line 329955
[15:01:34] ERROR: Could not sanitize molecule ending on line 329955


CPU times: user 3.77 s, sys: 40 ms, total: 3.81 s
Wall time: 3.81 s


In [4]:
coconut = pd.DataFrame(coco)

In [5]:
coconut.shape

(4371, 23)

Carregando datasets de antibióticos e comparando com o Coconut, usando InChIKey

In [6]:
antibio = pd.read_csv('/home/joao/documentos/IC-Joao/classificador/antibio.csv', sep=';', encoding='latin-1')

In [7]:
len(set(antibio['InChIKey']).intersection(set(coconut['inchikey'])))

0

In [8]:
antibio2 = pd.read_excel('/home/joao/documentos/IC-Joao/classificador/antibio2.xlsx')

In [9]:
antb2 = []

for x in antibio2['INCHI']:
    try:
        antb2.append(Chem.InchiToInchiKey(x))
    except:
        antb2.append('')


antibio2['InChIKey'] = antb2
antibio2 = antibio2[antibio2['InChIKey']!='']

In [10]:
len((set(coconut['inchikey'])).intersection(set(antibio2['InChIKey'])))

0

In [11]:
len((set(antibio2['InChIKey'])).intersection(set(antibio['InChIKey'])))


12

Juntando os dois conjuntos de dados

In [12]:
antibio2.columns

Index(['Name', 'SMILES', 'Molecular Formula', 'LogP', 'Molecular Weight',
       'HBD', 'HBA', 'TPSA', 'ROTB', 'AROM', 'ALERTS', 'INCHI', 'InChIKey'],
      dtype='object')

In [13]:
antibio2a = antibio2[['Name','SMILES','InChIKey']]

In [14]:
antibioa = antibio[['Name','SMILES','InChIKey']]

In [15]:
frames = [antibioa,antibio2a]
antibiotics_list = pd.concat(frames)

In [16]:
# Baixei a combinação para excluir as duplicatas manualmente
# from pathlib import Path  
# filepath = Path('/home/joao/documentos/IC-Joao/classificador/antibioticlist.xlsx')  
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# antibiotics_list.to_excel(filepath,index=False) 

In [17]:
#Criei uma variável com as duplicadas e seus indices para saber qual remover
dup = antibiotics_list['InChIKey'].duplicated()

In [18]:
# Criei um csv para poder visualizar todos os índices
# from pathlib import Path  
# filepath = Path('/home/joao/documentos/IC-Joao/classificador/dup.csv')  
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# dup.to_csv(filepath,index=True) 

Lista de antibióticos combinados, sem duplicatas



In [19]:
no_double_antib = pd.read_excel('/home/joao/documentos/IC-Joao/classificador/lista-antibioticos-sem-duplicatas.xlsx')

In [20]:
#Conferindo se não há duplicatas
y = 0
for x in no_double_antib['InChIKey'].duplicated():
    if x is True:
        y = y + 1
print(y)


0


In [21]:
no_double_antib.shape

(482, 3)

In [22]:
coconut.shape

(4371, 23)

In [23]:
#Falta verificar manualmente se todas as moléculas são efetivamente antibióticos

Adicionando coluna de 1 para indicar a classe de antibiótico

In [24]:
one = np.ones(482,dtype=int)
no_double_antib['Classe'] = one

Adicionando a coluna de 0 para indicar a classe de não antibiótico


In [25]:
zero = np.zeros(4371, dtype=int)
coconut['Classe'] = zero

Juntando os dados de antibióticos e não antibióticos

In [26]:
coconut.columns

Index(['coconut_id', 'inchi', 'inchikey', 'SMILES', 'sugar_free_smiles',
       'molecular_formula', 'molecular_weight', 'citationDOI', 'textTaxa',
       'name', 'synonyms', 'NPL_score', 'number_of_carbons',
       'number_of_nitrogens', 'number_of_oxygens', 'number_of_rings',
       'total_atom_number', 'bond_count', 'found_in_databases',
       'murko_framework', 'alogp', 'apol', 'topoPSA', 'Classe'],
      dtype='object')

In [27]:
coconut_renamed = coconut.rename(columns = {'name':'Name','inchikey':'InChIKey'})

In [28]:
coconut2 = coconut_renamed[['Name','SMILES','InChIKey','Classe']]

In [29]:
final_dataset = pd.concat([no_double_antib, coconut_renamed[['Name','SMILES','InChIKey','Classe']]])

In [30]:
from pathlib import Path  
filepath = Path('/home/joao/documentos/IC-Joao/classificador/final_data_classifier.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
final_dataset.to_csv(filepath,index=True)

In [31]:
new = pd.read_csv('/home/joao/documentos/IC-Joao/classificador/ligands_AntiBac.csv')

In [32]:
antibionew = new[['Name','SMILES','InChIKey']]

In [33]:
antibionew.shape

(305, 3)

In [34]:
one = np.ones(305,dtype=int)
antibionew['Classe'] = one

/home/joao/anaconda3/envs/diversedrl/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
antibionew.columns

Index(['Name', 'SMILES', 'InChIKey', 'Classe'], dtype='object')

In [36]:
final_dataset2 = pd.concat([antibionew, coconut_renamed[['Name','SMILES','InChIKey','Classe']]])

In [37]:
from pathlib import Path  
filepath = Path('/home/joao/documentos/IC-Joao/classificador/final_data_classifier2.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
final_dataset2.to_csv(filepath,index=True)

In [38]:
antimalar = pd.read_csv('/home/joao/documentos/IC-Joao/classificador/ligands_AntiMal.csv')

In [39]:
len((set(coconut['inchikey'])).intersection(set(antimalar['InChIKey'])))

0

In [40]:
antimalaria= antimalar[['Name','SMILES','InChIKey']]

In [41]:
one = np.ones(134,dtype=int)
antimalaria['Classe'] = one

/home/joao/anaconda3/envs/diversedrl/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [42]:
antimalaria_set = pd.concat([antimalaria, coconut_renamed[['Name','SMILES','InChIKey','Classe']]])

In [43]:
from pathlib import Path  
filepath = Path('/home/joao/documentos/IC-Joao/classificador/antimalaria_set.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
antimalaria_set.to_csv(filepath,index=True)

In [44]:
%%time

suppl = Chem.SDMolSupplier('coconut_antibiotic.sdf')

coco_antib = []

for mol in suppl:
    if type(mol) == Chem.rdchem.Mol:
        coco_antib.append(mol.GetPropsAsDict())

CPU times: user 224 ms, sys: 0 ns, total: 224 ms
Wall time: 220 ms


RDKit ERROR: [15:01:34] ERROR: Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [15:02:04] Explicit valence for atom # 10 N, 4, is greater than permitted
RDKit ERROR: [15:02:04] ERROR: Could not sanitize molecule ending on line 12391
[15:01:34] ERROR: Explicit valence for atom # 3 B, 4, is greater than permitted
[15:02:04] Explicit valence for atom # 10 N, 4, is greater than permitted
[15:02:04] ERROR: Could not sanitize molecule ending on line 12391
RDKit ERROR: [15:02:04] ERROR: Explicit valence for atom # 10 N, 4, is greater than permitted
[15:02:04] ERROR: Explicit valence for atom # 10 N, 4, is greater than permitted
[15:02:04] Explicit valence for atom # 5 N, 4, is greater than permitted
RDKit ERROR: [15:02:04] Explicit valence for atom # 5 N, 4, is greater than permitted
RDKit ERROR: [15:02:04] ERROR: Could not sanitize molecule ending on line 44741
[15:02:04] ERROR: Could not sanitize molecule ending on line 44741
RDKit ERROR: [15:02:04] ERROR: Explici

In [45]:
coconut_antib = pd.DataFrame(coco_antib)

In [46]:
coconut_antib.columns

Index(['coconut_id', 'SMILES'], dtype='object')

In [47]:
len(set(coconut['SMILES']).intersection(set(coconut_antib['SMILES'])))

0

In [48]:
coconut.tail()

,coconut_id,inchi,inchikey,SMILES,sugar_free_smiles,molecular_formula,molecular_weight,citationDOI,textTaxa,name,...,number_of_oxygens,number_of_rings,total_atom_number,bond_count,found_in_databases,murko_framework,alogp,apol,topoPSA,Classe
4366,CNP0004980,InChI=1S/C24H28FN5O/c1-14(2)19-9-16(10-23-29-2...,ZBJPKZONOXOXDW-UHFFFAOYSA-N,O=C(NCC1C=C(C)C(CC2=NC=3C=CC(F)=CC3N2)CC1C(C)C...,O=C(NCC1C=C(C)C(CC2=NC=3C=CC(F)=CC3N2)CC1C(C)C...,C24H28FN5O,421.511327,[],[notax],"N-({4-[(6-fluoro-1H-1,3-benzodiazol-2-yl)methy...",...,1,,59,34,[zincnp],n1ccnc(c1)CNCC2C=CC(Cc3nc4ccccc4[nH]3)CC2,1.2843,67.769204,78.21,0
4367,CNP0004981,InChI=1S/C23H30FN3O/c1-13(2)19-9-16(10-22-26-2...,UAVNLOQNSFKQBA-UHFFFAOYSA-N,O=C(NCC1C=C(C)C(CC2=NC=3C=CC(F)=CC3N2)CC1C(C)C...,O=C(NCC1C=C(C)C(CC2=NC=3C=CC(F)=CC3N2)CC1C(C)C...,C23H30FN3O,383.503066,[],[notax],"N-({4-[(6-fluoro-1H-1,3-benzodiazol-2-yl)methy...",...,1,,58,31,[zincnp],n1c2ccccc2[nH]c1CC3C=CC(CNCC4CC4)CC3,2.7660,65.142790,53.49,0
4368,CNP0004982,InChI=1S/C25H36FN3O/c1-6-17(7-2)25(30)27-14-19...,KDZDAAUYMMCLCJ-UHFFFAOYSA-N,O=C(NCC1C=C(C)C(CC2=NC=3C=CC(F)=CC3N2)CC1C(C)C...,O=C(NCC1C=C(C)C(CC2=NC=3C=CC(F)=CC3N2)CC1C(C)C...,C25H36FN3O,413.572182,[],[notax],"2-ethyl-N-({4-[(6-fluoro-1H-1,3-benzodiazol-2-...",...,1,,66,32,[zincnp],n1c2ccccc2[nH]c1CC3C=CCCC3,4.0500,72.663548,53.49,0
4369,CNP0004983,InChI=1S/C21H28FN3O/c1-12(2)18-8-15(13(3)7-16(...,DQNBOIXYERNBME-UHFFFAOYSA-N,O=C(NCC1C=C(C)C(CC2=NC=3C=CC(F)=CC3N2)CC1C(C)C)C,O=C(NCC1C=C(C)C(CC2=NC=3C=CC(F)=CC3N2)CC1C(C)C)C,C21H28FN3O,357.465713,[],[notax],"N-({4-[(6-fluoro-1H-1,3-benzodiazol-2-yl)methy...",...,1,,54,28,[zincnp],n1c2ccccc2[nH]c1CC3C=CCCC3,2.0082,60.289204,53.49,0
4370,CNP0004984,InChI=1S/C21H30N2O5/c1-14(2)11-19(25)22-16-5-3...,QMZPBPXBASHTDY-UHFFFAOYSA-N,O=C(NC1=CC=C(C=C1)C2OCC(=O)N(C3CCOCC3)C2CO)CC(C)C,O=C(NC1=CC=C(C=C1)C2OCC(=O)N(C3CCOCC3)C2CO)CC(C)C,C21H30N2O5,390.474107,[],[notax],N-{4-[3-(hydroxymethyl)-4-(oxan-4-yl)-5-oxomor...,...,5,,58,30,[zincnp],NaN,NaN,NaN,NaN,0


In [49]:
coconut_antib.head()

,coconut_id,SMILES
0,CNP0143582,O=C(O)C1=C(SCCNC(=O)C)CC2N1C(=O)C2C(O)C
1,CNP0222302,O=C(O)C1=C(SC=CN=C(O)C)CC2N1C(=O)C2C(O)C
2,CNP0150621,O=C(N(C)C1C(O)C(OC(=N)O)C(OC1NC2=NC3C(O)CN=C(O...
3,CNP0179118,OC(=NC1C(O)C(O)C(OC1CO)OC(C(O)CN)C(N)C(O)CO)CC
4,CNP0238626,N#CC1=CN(C=2N=CN=C(N)C12)C3OC(CO)C(O)C3O


In [50]:
coconut_antib.shape

(524, 2)

In [51]:
one1 = np.ones(524,dtype=int)
coconut_antib['Classe'] = one1

In [53]:
coconut_antib.head()

,coconut_id,SMILES,Classe
0,CNP0143582,O=C(O)C1=C(SCCNC(=O)C)CC2N1C(=O)C2C(O)C,1
1,CNP0222302,O=C(O)C1=C(SC=CN=C(O)C)CC2N1C(=O)C2C(O)C,1
2,CNP0150621,O=C(N(C)C1C(O)C(OC(=N)O)C(OC1NC2=NC3C(O)CN=C(O...,1
3,CNP0179118,OC(=NC1C(O)C(O)C(OC1CO)OC(C(O)CN)C(N)C(O)CO)CC,1
4,CNP0238626,N#CC1=CN(C=2N=CN=C(N)C12)C3OC(CO)C(O)C3O,1


In [54]:
finalset_for_real = pd.concat([coconut_antib, coconut_renamed[['SMILES','Classe']]])

In [ ]:
from pathlib import Path  
filepath = Path('/home/joao/documentos/IC-Joao/classificador/finale_ant.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
finalset_for_real.to_csv(filepath,index=True)